In [0]:
import pandas as pd
import numpy as np
from sklearn import datasets, linear_model, metrics 

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data = pd.read_csv('/content/drive/My Drive/cheese.csv')

In [0]:
data.head()

,RETAILER,VOLUME,DISP,PRICE
0,LOS ANGELES - LUCKY,21374,0.162000,2.578460
1,LOS ANGELES - RALPHS,6427,0.124113,3.727867
2,LOS ANGELES - VONS,17302,0.102000,2.711421
3,CHICAGO - DOMINICK,13561,0.027591,2.651206
4,CHICAGO - JEWEL,42774,0.090613,1.986674


Splitting the RETAILER column into COMPANY and CITY

In [0]:
reailer_list = list(data['RETAILER'])
city = []
company = []
for i in reailer_list:
    city.append(i.split('-')[0])
    company.append(i.split('-')[1])

In [0]:
data['CITY'] = city
data['COMPANY'] = company
data = data.drop('RETAILER', axis=1)
data.head()

,VOLUME,DISP,PRICE,CITY,COMPANY
0,21374,0.162000,2.578460,LOS ANGELES,LUCKY
1,6427,0.124113,3.727867,LOS ANGELES,RALPHS
2,17302,0.102000,2.711421,LOS ANGELES,VONS
3,13561,0.027591,2.651206,CHICAGO,DOMINICK
4,42774,0.090613,1.986674,CHICAGO,JEWEL


EDA

In [0]:
data['COMPANY'].describe()

count            5555
unique             50
top        WINN DIXIE
freq              723
Name: COMPANY, dtype: object

In [0]:
data['CITY'].describe()

count           5555
unique            46
top       CHARLOTTE 
freq             244
Name: CITY, dtype: object

### Estimating the Demand curve

We use the followig linear regression model to estimate the demand curve :
\begin{equation}
  volume = \alpha + \beta*price + \epsilon
\end{equation}
where, $\alpha$ and $\beta$ are the intercept and slope of the demand curve respectively.

In [0]:
# function to generate demand curve company/city-wise

company_dict = {}
city_dict = {}

def demand_curve_city(city_name):
  for i in set(city):
    y = np.array(data.loc[data['CITY'] == i]['VOLUME'])
    y = y.reshape(len(y),1)
    x = np.array(data.loc[data['CITY'] == i]['PRICE'])
    x = x.reshape(len(x),1)
    reg = linear_model.LinearRegression() 
    reg.fit(x, y) 
    city_dict[i] = [round(reg.intercept_[0],2), round(reg.coef_[0][0],2)]
  for i in set(city):
    if city_name in i:
      print('The demand curve for ' + str(i) + ' city is : \n')
      print('Volume = ' + str(round(reg.intercept_[0],2)) + str(round(reg.coef_[0][0],2)) + ' * Price'  )

def demand_curve_company(company_name):
  for i in set(company):
    y = np.array(data.loc[data['COMPANY'] == i]['VOLUME'])
    y = y.reshape(len(y),1)
    x = np.array(data.loc[data['COMPANY'] == i]['PRICE'])
    x = x.reshape(len(x),1)
    reg = linear_model.LinearRegression() 
    reg.fit(x, y) 
    company_dict[i] = [round(reg.intercept_[0],2), round(reg.coef_[0][0],2)]
  for i in set(company):
    if company_name in i:
      print('The demand curve for ' + str(i) + ' company is : \n')
      print('Volume = ' + str(round(reg.intercept_[0],2)) + str(round(reg.coef_[0][0],2)) + ' * Price'  )

In [0]:
# An example demonstrating the demand curve generating function
demand_curve_city('SACRAMENTO')

The demand curve for SACRAMENTO  city is : 

Volume = 9541.5-2740.36 * Price


In [0]:
# An example demonstrating the demand curve generating function
demand_curve_company('LUCKY')

The demand curve for  LUCKY company is : 

Volume = 8051.99-1777.61 * Price


<br>

### Computing profitability city/retailer-wise using the estimated emand curves

Now using the demand curves we predict the voulme for each city/company for the discounted price (price reduced by 5 %). We use the following assumptions and equations.
- Discounted price : Price reduced by 5%
- Unit cost : \$ 1.8 per unit
- $new\_volume$ = volume estimated using the demand curve
- $old\_revenue = old\_volume * old\_price$
- $new\_revenue = new\_volume * discounted\_price$

\begin{equation}
  profitability = new\_profit - old\_profit
\end{equation}

In [0]:
data['DISC_PRICE'] = data['PRICE']*0.95

In [0]:
new_vol_city = []
for ind in data.index: 
    new_vol_city.append(city_dict[data['CITY'][ind]][0] + city_dict[data['CITY'][ind]][1]*data['DISC_PRICE'][ind])

new_vol_company = []
for ind in data.index: 
    new_vol_company.append(company_dict[data['COMPANY'][ind]][0] + company_dict[data['COMPANY'][ind]][1]*data['DISC_PRICE'][ind])    

In [0]:
data['REVENUE'] = data['PRICE'] * data['VOLUME']
data['COST'] = data['VOLUME'] * 1.8
data['PROFIT'] = data['REVENUE'] - data['COST']
data['NEW_VOL_CITY'] = new_vol_city
data['NEW_VOL_COMPANY'] = new_vol_company
data['NEW_REV_CITY'] = data['NEW_VOL_CITY'] * data['DISC_PRICE']
data['NEW_REV_COMPANY'] = data['NEW_VOL_COMPANY'] * data['DISC_PRICE']
data['NEW_COST_COMPANY'] = data['NEW_VOL_COMPANY'] * 1.8
data['NEW_COST_CITY'] = data['NEW_VOL_CITY'] * 1.8
data['NEW_PROFIT_CITY'] = data['NEW_REV_CITY'] - data['NEW_COST_CITY'] 
data['NEW_PROFIT_COMPANY'] = data['NEW_REV_COMPANY'] - data['NEW_COST_COMPANY']
data['PROFITABILITY_CITY'] = data['NEW_PROFIT_CITY'] - data['PROFIT']
data['PROFITABILITY_COMPANY'] = data['NEW_PROFIT_COMPANY'] - data['PROFIT']
data['PROFITABILITY_CITY_%'] = data['PROFITABILITY_CITY']/data['PROFIT']*100
data['PROFITABILITY_COMPANY_%'] = data['PROFITABILITY_COMPANY']/data['PROFIT']*100

In [0]:
data.head()

,VOLUME,DISP,PRICE,CITY,COMPANY,DISC_PRICE,REVENUE,COST,PROFIT,NEW_VOL_CITY,NEW_VOL_COMPANY,NEW_REV_CITY,NEW_REV_COMPANY,NEW_COST_COMPANY,NEW_COST_CITY,NEW_PROFIT_CITY,NEW_PROFIT_COMPANY,PROFITABILITY_CITY,PROFITABILITY_COMPANY,PROFITABILITY_CITY_%,PROFITABILITY_COMPANY_%
0,21374,0.162000,2.578460,LOS ANGELES,LUCKY,2.449537,55112.004040,38473.2,16638.804040,16301.282031,14693.893175,39930.593482,35993.235006,26449.007715,29342.307655,10588.285826,9544.227291,-6050.518214,-7094.576749,-36.363901,-42.638742
1,6427,0.124113,3.727867,LOS ANGELES,RALPHS,3.541474,23959.001209,11568.6,12390.401209,6524.485283,3248.338480,23106.292710,11503.905133,5847.009264,11744.073510,11362.219200,5656.895869,-1028.182009,-6733.505340,-8.298214,-54.344530
2,17302,0.102000,2.711421,LOS ANGELES,VONS,2.575850,46913.006142,31143.6,15769.406142,15170.322612,17749.564151,39076.474742,45720.213931,31949.215472,27306.580702,11769.894040,13770.998459,-3999.512102,-1998.407683,-25.362478,-12.672688
3,13561,0.027591,2.651206,CHICAGO,DOMINICK,2.518646,35953.004566,24409.8,11543.204566,22115.488336,28599.932348,55701.079600,72033.096629,51479.878227,39807.879004,15893.200596,20553.218402,4349.996030,9010.013836,37.684475,78.054701
4,42774,0.090613,1.986674,CHICAGO,JEWEL,1.887340,84977.993676,76993.2,7984.793676,38125.620549,69084.140406,71956.020126,130385.282278,124351.452730,68626.116989,3329.903136,6033.829548,-4654.890540,-1950.964128,-58.296942,-24.433495


The following dataframe stores the estimated Profit after the price is educed by 5%, corresponding to each CITY-RETAILER pair using both the **Retailer based** and **Location based** demand curves.

In [0]:
data_results = pd.DataFrame()
data_results['CITY'] = data['CITY']
data_results['COMPANY'] = data['COMPANY']
data_results['PROFITABILITY_CITY_%'] = data['PROFITABILITY_CITY']/data['PROFIT']*100
data_results['PROFITABILITY_COMPANY_%'] = data['PROFITABILITY_COMPANY']/data['PROFIT']*100

In [0]:
data_results.head()

,CITY,COMPANY,PROFITABILITY_CITY_%,PROFITABILITY_COMPANY_%
0,LOS ANGELES,LUCKY,-36.363901,-42.638742
1,LOS ANGELES,RALPHS,-8.298214,-54.344530
2,LOS ANGELES,VONS,-25.362478,-12.672688
3,CHICAGO,DOMINICK,37.684475,78.054701
4,CHICAGO,JEWEL,-58.296942,-24.433495


Now, to answer the question <br>
Which Retailer's profitability increase or decrease due to 5% discounting in price ?? <br>
- we use the **Company** model of demand curve and aggregate the estimated profitability for each company/retailer irrespective of the location.

In [0]:
# This code creates a dictionary which stores the average percentage change in profitability for each company/retailer.

company_profitability = {}
for i in set(company):
  company_profitability[i] = [np.average(data_results['PROFITABILITY_COMPANY_%'].loc[data_results['COMPANY'] == i]), set(data_results['CITY'].loc[data_results['COMPANY'] == i])]

In [0]:
print('The top 10 companies/retailers (along with the avg profitability %) whose profitability increased the most in the discount scheme :\n ')
c = []
p = []
l = [] 

for i in sorted(company_profitability.items(), key=lambda x: x[1], reverse = True)[:10]:
  c.append(i[0])
  p.append(i[1][0])
  l.append(i[1][1])

# Create DataFrame 
data = {'COMPANY':c, 'AVG_%_PROFITABILITY':p, 'CITIES':l} 
df = pd.DataFrame(data)  
df

The top 10 companies/retailers (along with the avg profitability %) whose profitability increased the most in the discount scheme :
 


,COMPANY,AVG_%_PROFITABILITY,CITIES
0,TOPS MARKETS,132.584249,{BUFFALO/ROCHESTER }
1,WEGMANS,105.602246,"{SYRACUSE , BUFFALO/ROCHESTER }"
2,P & C FOOD MARKE,92.968279,{SYRACUSE }
3,PATHMARK,91.926375,{NEW YORK (NEW) }
4,PRICE CHOPPER,76.472074,"{SYRACUSE , ALBANY,NY }"
5,STAR MARKET,73.130014,{BOSTON }
6,DOMINICK,52.051722,{CHICAGO }
7,RALPHS,47.470044,"{SAN DIEGO , LOS ANGELES }"
8,PUBLIX,45.293928,"{TAMPA/ST. PETE , ORLANDO,FL , MIAMI , JACKSON..."
9,OMNI,43.968344,{CHICAGO }


In [0]:
print('The bottom 10 companies/retailers (along with the in avg profitability %) whose profitability decreased the most in the discount scheme :\n ')
c = []
p = []
l = []

for i in sorted(company_profitability.items(), key=lambda x: x[1])[:10]:
  c.append(i[0])
  p.append(i[1][0])
  l.append(i[1][1])

# Create DataFrame 
data = {'COMPANY':c, 'AVG_%_PROFITABILITY':p, 'CITIES':l} 
df = pd.DataFrame(data)  
df

The bottom 10 companies/retailers (along with the in avg profitability %) whose profitability decreased the most in the discount scheme :
 


,COMPANY,AVG_%_PROFITABILITY,CITIES
0,SHOP N SAVE,-23.534963,{NEW ENGLAND (NORTH) }
1,BI LO,-22.585993,"{SOUTH CAROLINA , CHARLOTTE }"
2,KROGER,-18.127992,{BIRMINGHAM/MONTGOM }
3,BRUNOS,-14.254429,{BIRMINGHAM/MONTGOM }
4,HARRIS TEETER,-11.070944,{CHARLOTTE }
5,NEW FARM FRESH,-8.377674,{RICHMOND/NORFOLK }
6,GIANT FOOD STO,-7.069188,{HARRISBURG/SCRANTN }
7,KASH N KARRY,-6.992935,{TAMPA/ST. PETE }
8,HARVEST FOODS,-3.815757,{LITTLE ROCK }
9,ALBERTSONS,-2.503561,{DALLAS/FT. WORTH }


From the results abtained above we observe the following:<br>
- The discount scheme (discountig the price by 5%) has resulted in a increase in demand for almost all CITIES and for almost all RETAILERS/CONPANIES. Since all the demand curves have a **-ve** slope both city/company-wise.
- The maximum profitability observed is for the company *TOPS MARKETS* with average percentage of profitability 132.584249 %, which implies almost 1.3 times increase in profit!
- The minimum profitability observed is for the company *SHOP N SAVE* with average percentage of profitability -23.534963 %, which implies almost .2 times drop in profit!



We separate the 2 groups of retailers with **positive** and **negative** profitability, as follows:

In [0]:
positive_profitability_company = []
negative_profitability_company = []
for i in company_profitability.keys():
  if company_profitability[i][0] > 0:
    positive_profitability_company.append(i)
  else:
    negative_profitability_company.append(i)  

In [0]:
# Companies with positive profitability
positive_profitability_company

[' JEWEL',
 ' PUBLIX',
 ' SAFEWAY',
 ' DOMINICK',
 ' BIG BEAR',
 ' SMITHS FOOD',
 ' NATIONAL SUPER',
 ' KING SOOPERS INC',
 ' GIANT FOOD INC',
 ' H E BUTT',
 ' SHAWS',
 ' STOP N SHOP',
 ' TOM THUMB',
 ' A & P',
 ' TOPS MARKETS',
 ' ACME MARKET',
 ' HOMELAND',
 ' RANDALLS',
 ' P & C FOOD MARKE',
 ' PRICE CHOPPER',
 ' PATHMARK',
 ' RALEYS',
 ' KOHLS FOOD STORE',
 ' LUCKY',
 ' VONS',
 ' KROGER CO',
 ' FARMER JACKS',
 ' WEGMANS',
 ' OMNI',
 " FRY'S FOOD STORE",
 ' SCHNUCK MARKETS',
 ' FOOD LION',
 ' WALDBAUMS',
 ' STAR MARKET',
 ' STOP & SHOP',
 ' RALPHS',
 ' GIANT EAGLE',
 ' WINN DIXIE',
 ' SUPER FRESH']

In [0]:
# Companies with negative profitability
negative_profitability_company

[' GIANT FOOD STO',
 ' KASH N KARRY',
 ' BRUNOS',
 ' NEW FARM FRESH',
 ' ALBERTSONS',
 ' HARRIS TEETER',
 ' BI LO',
 ' KROGER',
 ' HARVEST FOODS',
 ' SHOP N SAVE',
 ' DILLON COMPANIES']

Till now we have done the analysis based on the company model
- we use the **City** model of demand curve and aggregate the estimated profitability for each city irrespective of the retailer.
<br>
The motive of doing this is : Suppose we want to find out the city(ies) where it is most profitable to launch this discount scheme. Here we assume that people's behaviour towards the product is dependent on their location of stay and irrespective of the brands.

In [0]:
# This code creates a dictionary which stores the average percentage change in profitability for each city.

city_profitability = {}
for i in set(city):
  city_profitability[i] = [np.average(data_results['PROFITABILITY_CITY_%'].loc[data_results['CITY'] == i]), set(data_results['COMPANY'].loc[(data_results['CITY'] == i) & (data_results['PROFITABILITY_CITY_%'] > 0)]) ]

In [0]:
print('The top 10 cities (along with the avg % profitability) where profitability increased the most in the discount scheme :\n ')
c = []
p = []

for i in sorted(city_profitability.items(), key=lambda x: x[1], reverse = True)[:10]:
  c.append(i[0])
  p.append(i[1][0])

# Create DataFrame 
data = {'CITY':c, 'AVG_%_PROFITABILITY':p} 
df = pd.DataFrame(data)  
df

The top 10 cities (along with the avg % profitability) where profitability increased the most in the discount scheme :
 


,CITY,AVG_%_PROFITABILITY
0,CHICAGO,133.835720
1,SYRACUSE,107.594319
2,BUFFALO/ROCHESTER,97.276887
3,NEW YORK (NEW),85.127574
4,BALTI/WASH,58.598519
5,"ALBANY,NY",49.331276
6,PHILADELPHIA,43.574180
7,HARTFORD,40.415985
8,BIRMINGHAM/MONTGOM,31.225523
9,ST. LOUIS,31.138204


In [0]:
print('The bottom 10 cities (along with the avg % profitability) where profitability decreased the most in the discount scheme :\n ')
c = []
p = []

for i in sorted(city_profitability.items(), key=lambda x: x[1])[:10]:
  c.append(i[0])
  p.append(i[1][0])
  

# Create DataFrame 
data = {'CITY':c, 'AVG_%_PROFITABILITY':p} 
df = pd.DataFrame(data)  
df

The bottom 10 cities (along with the avg % profitability) where profitability decreased the most in the discount scheme :
 


,CITY,AVG_%_PROFITABILITY
0,NEW ENGLAND (NORTH),-23.534963
1,NASHVILLE,-17.524173
2,MIAMI,-13.634372
3,HARRISBURG/SCRANTN,-7.069188
4,CHARLOTTE,-6.423914
5,INDIANAPOLIS,-6.203457
6,NEW ORLEANS,-4.718886
7,LITTLE ROCK,-3.815757
8,TAMPA/ST. PETE,-2.795625
9,ROANOKE (NEW),-1.194958


It is worth noticing that the cities corresponding the top 10 high profitability companies are also among the top 10 cities where the profitability is high and vice versa.